In [1]:
import requests
import pyarrow as pa
import pyarrow.fs
import io


In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 16293470208 (15.17 GB)
DFS Remaining: 16293421056 (15.17 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.4:9866 (project-4-p4_yw_ym-dn-1.project-4-p4_yw_ym_default)
Hostname: 595de693472a
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 17657540608 (16.44 GB)
DFS Remaining: 8146710528 (7.59 GB)
DFS Used%: 0.00%
DFS Remaining%: 31.55%
Conf

In [3]:
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

--2024-03-14 23:09:16--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv.13’

hdma-wi-2021.csv.13 100%[===================>] 166.84M   123MB/s    in 1.4s    

2024-03-14 23:09:17 (123 MB/s) - ‘hdma-wi-2021.csv.13’ saved [174944099/174944099]



In [4]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [5]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/single.csv
!hdfs dfs -du -h hdfs://boss:9000/double.csv

166.8 M  166.8 M  hdfs://boss:9000/single.csv
166.8 M  333.7 M  hdfs://boss:9000/double.csv


In [6]:
#q3
!curl -i "http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"

HTTP/1.1 200 OK
Date: Thu, 14 Mar 2024 23:09:53 GMT
Cache-Control: no-cache
Expires: Thu, 14 Mar 2024 23:09:53 GMT
Date: Thu, 14 Mar 2024 23:09:53 GMT
Pragma: no-cache
X-Content-Type-Options: nosniff
X-FRAME-OPTIONS: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Content-Type: application/json
Transfer-Encoding: chunked

{"FileStatus":{"accessTime":1710457763738,"blockSize":1048576,"childrenNum":0,"fileId":16386,"group":"supergroup","length":174944099,"modificationTime":1710457772795,"owner":"root","pathSuffix":"","permission":"644","replication":1,"storagePolicy":0,"type":"FILE"}}

In [7]:
#q4
!curl -i -L "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true"

HTTP/1.1 200 OK
Date: Thu, 14 Mar 2024 23:09:53 GMT
Cache-Control: no-cache
Expires: Thu, 14 Mar 2024 23:09:53 GMT
Date: Thu, 14 Mar 2024 23:09:53 GMT
Pragma: no-cache
X-Content-Type-Options: nosniff
X-FRAME-OPTIONS: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Content-Type: application/json
Transfer-Encoding: chunked

{"Location":"http://43dc2ad30e77:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0"}

In [8]:
#q5
url = "http://boss:9870/webhdfs/v1/single.csv?op=GET_BLOCK_LOCATIONS&length="
response = requests.get(url)
data = response.json()

block_distribution = {}
for block in data['LocatedBlocks']['locatedBlocks']:
    for location in block['locations']:
        #print(location)
        datanode_id = location['hostName']  
        #print(datanode_id)
        block_distribution[datanode_id] = block_distribution.get(datanode_id, 0) + 1

print(block_distribution)

{'43dc2ad30e77': 76, '595de693472a': 91}


In [9]:
#q6
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
file_path = '/single.csv'  
f = hdfs.open_input_file(file_path)
f.read_at(10, 0)


2024-03-14 23:09:55,243 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [10]:
#q7
with hdfs.open_input_file(file_path) as file_obj:
    reader = io.TextIOWrapper(io.BufferedReader(file_obj))
    #print(reader)
    count = 0
    for i in reader:
        if "Single Family" in i:
            count +=1
count


444874